# Clojupyter - Clojure in Jupyter Lab, Notebook, and Console

This notebook demonstrates some of the features of Clojupyter.  Things that are shown here should work in both Jupyter Lab *and* Jupyter Notebook (and some of it in Jupyter Console as well, but since Clojure has the REPL we usually don't think very much about Console).

Jupyter Lab and Jupyter Notebook are different in some respects, this notebook only shows features that work in both.  There are separate demo notebooks showing clojupyter's support for features that are specific to either Lab or Notebook.

## Basics: Evaluating Clojure expressions

Clojupyter basically lets you evaluate Clojure expressions from a Jupyter notebook, at its most fundament it is not unlike a regular Clojure REPL:

In [ ]:
(time (reduce + (range 1000)))

Note above that we both see the side-effects of printing *and* the result of evaluating the expression.  Let's see which version of Clojure and clojupyter we're using:

In [ ]:
(println (str "Date:\t\t\t" (java.util.Date.)))
(println (apply format "Clojure version:\tv%d.%d" ((juxt :major :minor) *clojure-version*)))
(println (str "Clojupyter version:\t" clojupyter/version))
;

The **semicolon** at the end of the cell tells clojupyter not to print the result of evaluating the cell which is useful when we're only interested in side-effects such as loading libraries. In this instance the semicolon appears on its own line which is not necessary, as long as the semicolon is the last non-whitespace character it will have the effect of suppressing printing of the cell evaluation result.

## Rich content at your fingertips

### Hiccup

We can use [Hiccup](https://github.com/weavejester/hiccup) to render HTML.  To do this conveniently, we first add convenient access to the `clojupyter.display` namespace usng the alias `display`:

In [ ]:
(require '[clojupyter.display :as display])

which gives us convenient access to generating formatted output using HTML:

In [ ]:
;; displaying html
(display/hiccup
    [:ul 
     [:li "an " [:i "emphatic"] " idea"]
     [:li "a " [:b "bold"] " idea"]
     [:li "an " [:span {:style "text-decoration: underline;"} "important"] " idea"]])

which works for Scalable Vector Graphics (SVG) as well:

In [ ]:
(display/hiccup
    [:svg {:height 100 :width 100 :xmlns "http://www.w3.org/2000/svg"}
            [:circle {:cx 50 :cy 40 :r 40 :fill "red"}]])

### HTML

We can also render html strings directly:

In [ ]:
(display/html "<p style=\"color:red\">Hello world!</p>")

### Markdown

In [ ]:
(display/markdown "
**TO DO:**                  
* [X] Install **Clojupyter**.
* [ ] Build something *cool*.
* [ ] ???
* [ ] Profit!")

Overall, we have very direct access to controlling what is displayed by Jupyter:

In [ ]:
(display/render-mime "text/plain" "This is plain text.")

In [ ]:
(display/render-mime "text/html" "<h1>This is a heading</h1>")

And we have all the facilities of Clojure at our disposal for generating the content in the notebook!

### Images

We also have direct access to displaying bitmaps, here a clokupyter logo:

In [ ]:
(->> clojupyter/logo type (str "Logo is of type: ") println)
clojupyter/logo

Helper function to read bytes from uri string

In [ ]:
(defn slurp-bytes
  "Slurp the bytes from a slurpable thing"
  [x]
  (with-open [out (java.io.ByteArrayOutputStream.)]
    (clojure.java.io/copy (clojure.java.io/input-stream x) out)
    (.toByteArray out)))

GIF

In [ ]:
(-> "https://upload.wikimedia.org/wikipedia/commons/d/d3/Newtons_cradle_animation_book_2.gif"
    slurp-bytes
    display/gif)

We have no custom converters for all image types, but custom ones can be created using `clojure.display/render-mime`. To render the image correctly, we need to convert the byte array to a base64 encoded string.

In [ ]:
(display/render-mime :image/png
   (-> "../resources/clojupyter/assets/logo-64x64.png"
       slurp-bytes
       clojure.data.codec.base64/encode
       String.))

## Using external Clojure libraries

You can fetch external Clojure dependencies using `add-dependencies` in the namespace `clojupyter.misc.helper`:

In [ ]:
(require '[clojupyter.misc.helper :as helper])
(helper/add-dependencies '[incanter "1.5.7"])
(use '(incanter core stats charts io)) ; include Incanter's facilities into working namespace
:ok

## Example: Plotting using Incanter

As shown above, clojupyter display bitmaps directly when [BufferedImage](https://docs.oracle.com/javase/7/docs/api/java/awt/image/BufferedImage.html) are returned. This makes it easy to use Java charting libraries, such as [Incanter](https://github.com/incanter/incanter), where charts are easily converted into a bitmaps. Since Incanter simply wraps the Java charting library [JFreeChart](https://github.com/incanter/incanter), we can call 
`(.createBufferedImage chart width height)` on any Incanter chart to get an image we can render as cell output:

In [ ]:
(-> (sample-normal 10000)
    histogram
    (.createBufferedImage 600 400))

Here's an example of a scatter plot:

In [ ]:
(-> (scatter-plot (sample-normal 1000) 
                  (sample-normal 1000)
                  :x-label "x" :y-label "y")
    (.createBufferedImage 600 400)) 

And with a very liberal license, you can do pretty much whatever you want with clojupyter:

In [ ]:
clojupyter/license

**We hope you'll enjoy clojupyter!**